In [ ]:
!pip3 install transformers codecarbon datasets trl

In [2]:
from codecarbon import EmissionsTracker
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T", torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [4]:
tracker = EmissionsTracker(output_file="./test.csv")

[codecarbon INFO @ 15:58:50] [setup] RAM Tracking...
[codecarbon INFO @ 15:58:50] [setup] CPU Tracking...
[codecarbon WARNING @ 15:58:50] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 15:58:51] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:58:51] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:58:51] [setup] GPU Tracking...
[codecarbon INFO @ 15:58:51] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:58:51] >>> Tracker's metadata:
[codecarbon INFO @ 15:58:51]   Platform system: Linux-6.6.56+-x86_64-with-glibc2.35
[codecarbon INFO @ 15:58:51]   Python version: 3.10.12
[codecarbon INFO @ 15:58:51]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 15:58:51]   Available RAM : 31.351 GB
[codecarbon INFO @ 15:58:51]   CPU count: 4
[codeca

In [5]:
user_message = "Don't forget to like and subscribe!"
inputs = tokenizer(user_message, return_tensors="pt")

tracker.start()
outputs = model.generate(**inputs.to("cuda"),
                         max_new_tokens=12,
                         num_beams=10,
                         early_stopping=True,
                         no_repeat_ngram_size=2)
tracker.stop()

result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result[0])

[codecarbon INFO @ 15:58:56] Energy consumed for RAM : 0.000005 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 15:58:56] Energy consumed for all CPUs : 0.000019 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:58:56] Energy consumed for all GPUs : 0.000037 kWh. Total GPU Power : 81.30649897836354 W
[codecarbon INFO @ 15:58:56] 0.000062 kWh of electricity used since the beginning.


Don't forget to like and subscribe! https://www.youtube.com/watch?v=


In [1]:
from datasets import load_dataset
from transformers import TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer, SFTConfig
import os
import torch

model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T", torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
os.environ["WANDB_DISABLED"] = "true"

dataset = load_dataset("tatsu-lab/alpaca", split="train[:1%]").select(range(20))

training_params = SFTConfig(
    output_dir="./model_train",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_params,
)

trainer.train()

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-1-af90cc280fc9>:22: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

[codecarbon INFO @ 15:55:15] [setup] RAM Tracking...
[codecarbon INFO @ 15:55:15] [setup] CPU Tracking...
[codecarbon WARNING @ 15:55:15] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 15:55:16] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:55:16] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 15:55:16] [setup] GPU Tracking...
[codecarbon INFO @ 15:55:16] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:55:16] >>> Tracker's metadata:
[codecarbon INFO @ 15:55:16]   Platform system: Linux-6.6.56+-x86_64-with-glibc2.35
[codecarbon INFO @ 15:55:16]   Python version: 3.10.12
[codecarbon INFO @ 15:55:16]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 15:55:16]   Available RAM : 31.351 GB
[codecarbon INFO @ 15:55:16]   CPU count: 4
[codeca

Step,Training Loss


[codecarbon INFO @ 15:55:34] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 15:55:34] Energy consumed for all CPUs : 0.000178 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:55:34] Energy consumed for all GPUs : 0.000401 kWh. Total GPU Power : 95.59812257181933 W
[codecarbon INFO @ 15:55:34] 0.000629 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:55:41] Energy consumed for RAM : 0.000071 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 15:55:41] Energy consumed for all CPUs : 0.000255 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 15:55:41] Energy consumed for all GPUs : 0.000511 kWh. Total GPU Power : 60.56832324753425 W
[codecarbon INFO @ 15:55:41] 0.000837 kWh of electricity used since the beginning.
/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will 

TrainOutput(global_step=20, training_loss=1.1072583198547363, metrics={'train_runtime': 21.6968, 'train_samples_per_second': 0.922, 'train_steps_per_second': 0.922, 'total_flos': 15101811781632.0, 'train_loss': 1.1072583198547363, 'epoch': 1.0})